In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

import warnings
warnings.filterwarnings('ignore')

from flask import Flask
from flask import request
from flask import render_template
import mysql.connector
from mysql.connector import Error
import string

In [44]:
# Function to tokenize the episode names
# def token(s): 
#     getVals = list([val for val in s
#                 if val.isalpha() or val.isnumeric()])

#     result = "".join(getVals)
#     result = result.lower()
#     return result



def token(text):
    # Remove everything except normal alphabets and numbers
    cleaned_text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Tokenize the cleaned text
    tokens = cleaned_text.split()
    result =""
    for a in tokens:
        result = result + a
    result = result.lower()
    return result

In [45]:
s="farewellôçª"
token(s)

'farewell'

In [46]:
# Test Data

#v_id = ['52c87a97-0663-4188-83b0-8d1db0eae6ec']
v_id = ['96932f06-f7e1-4aa7-b84d-b96e5b32e999',
# '5008fdb1-0c66-46e7-8ecf-cbe03021f950',
#'0707deae-17b9-4ec1-b45d-56532f320f86',
'946fbd54-d3fd-4e66-b5a1-fac18d6305ed',
'52c87a97-0663-4188-83b0-8d1db0eae6ec',
'92e510f1-9ce2-478b-a93d-8e40fd0ca529',
'f9e1fe71-c91a-4ddb-9371-937e40e18ec9',
'720b5551-e62b-4e08-80f7-2067e7151673',
'097c473f-3598-48eb-b303-d822c3fe0206',
'e016138e-bb89-4179-82c1-d60e4e647b72',
'2b6e5d08-758d-4837-abda-b46ba6644bcd',
'c6236338-f69c-44fa-9d41-7c273bea5ab7',
'7c3cc2de-3ff6-48a2-b508-bd9b00f4a61f',
'2f9f6b2c-7bb1-48fa-baaf-cfea2ab20602',
'ffcb664c-9c70-4e1a-bda7-e8d98382178d',
'5a6923c7-fffd-4862-b7f8-949bcd504a50',
'd8aeab3c-56a9-461a-9fea-82d02a35a1fd',
'dd8d7f63-54d2-4968-b5b7-40d8ffe75531',
'c0da4f8d-483b-463a-92ef-7ed97f382221',
'932b94fd-7769-4ebd-8e5c-273e70961c8c',
'850c08ec-28f2-4feb-bb82-f87ba5b3dfdd',
'2ec1fe84-92d9-4080-92e4-b52dda7c518b',
'cd13c993-7b0a-4113-8cb3-c3bf48f91fc0',
#'a7beff11-d9e5-42e4-af88-8f96a30fdec8',
'ee617385-353d-42d8-83ed-44097c368512']

In [47]:
# Connect to MYSQL Database
connection = mysql.connector.connect(host='winv-ahd-003',
            database='vonly_data_feed_us_staging',
            user='vonly-agent',
            password='a714fded-311c-4215-8b8b-5df4086e264b')

In [49]:
test_case=3
for id_1 in v_id:
    print("\n","Test Case: ",test_case)
    #print(id_1)
    test_case=test_case+1
    
    # SQL Query
    sql_select_Query = """  
    SELECT  te.platform,te.portal,te.vonly_asset_id ,ts.title 'season_name', ts.num 'season_no',ts.portal_item_id 'season_portal_item_id', 
    te.title 'episode_name',te.num 'episode_no' ,te.portal_item_id  'episode_portal_item_id' 
    FROM tv_id_mappings tim
    INNER JOIN tv_ids te ON te.id=tim.episode_vonly_id
    INNER JOIN tv_ids ts ON ts.id=tim.season_vonly_id
    WHERE tim.season_vonly_id IN (
    SELECT id FROM tv_ids WHERE scope='tv_season' AND vonly_asset_id='{}');""".format(id_1)   

    cursor = connection.cursor()        
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
#     for r in records:
#         print(r[0], r[7], r[6])
    
    # Dictionaries for each portal and one general to compare where for each keys is episode number and value is apisode name
    amazon = dict()
    vudu = dict()
    google = dict()
    apple = dict()
    itunes = dict()
    general = dict()
    
    
    # Code to fill the above dictionaries, skipping the trailers
    for r in records:
        platform = r[0]
        episode_no = r[7]
        episode_name = r[6]
        if(platform == 'iTunes' ):
            if(int(episode_no)>50 or int(episode_no)==0): 
                continue   
            itunes[episode_no] = episode_name
            
        elif(platform == 'VUDU' ):
            if(int(episode_no)>50 or int(episode_no)==0): 
                continue
            vudu[episode_no] = episode_name
            
        elif(platform == 'Amazon Prime Video' ):
            if(int(episode_no)>50 or int(episode_no)==0): 
                continue
            amazon[episode_no] = episode_name

        elif(platform == 'AppleTVApp' ):
            if(int(episode_no)>50 or int(episode_no)==0): 
                continue
            apple[episode_no] = episode_name
            
        elif(platform == 'Google Play' ):
            if(int(episode_no)>50 or int(episode_no)==0): 
                continue
            google[episode_no] = episode_name
            
        else:
            print("Portal not in the list")
    
    
    # Checking if any portal is not present totally
    amazon_class=itunes_class=appletv_class=vudu_class=google_class=0
    if(len(amazon) == 0 ):
        amazon_class=1
        print("Amazon portal is not there.")

    if(len(itunes) == 0):
        itunes_class=1
        print("iTunes portal is not there.")

    if(len(apple) == 0):
        appletv_class=1
        print("Apple portal is not there.")

    if(len(vudu) == 0):
        vudu_class=1
        print("Vudu portal is not there.")

    if(len(google) == 0):
        google_class=1
        print("Google portal is not there.")

    
    # Check whether each portal have same number of episodes or not
    x=0
    len_= max(len(itunes),len(apple),len(google),len(amazon),len(vudu))
    print("len=",len_)
    if(itunes_class==0):
        if(len(itunes) == len_):
            x=x+1
        else:
            print("iTunes length is not same")

    if(appletv_class==0):
        if(len(apple) == len_):
            x=x+1
        else:
            print("Apple length is not same")
    
    if(amazon_class==0):
        if(len(amazon) == len_):
            x=x+1
        else:
            print("Amazon length is not same")

    if(google_class==0):    
        if(len(google) == len_):
            x=x+1
        else:
            print("Google length is not same")
    
    if(vudu_class==0):
        if(len(vudu) == len_):
            x=x+1
        else:
            print("Vudu length is not same")
        


    #print(x) 
    if(x!=5-(amazon_class+itunes_class+appletv_class+vudu_class+google_class)):
        print("\033[1mNot Properly Mapped Due to Size\033[0m")
        continue
    
#     print(len(itunes))
#     print(len(apple))
#     print(len(amazon))
#     print(len(vudu))
#     print(len(google))
        
         
#     print("a",amazon,"\n")
#     print("i",itunes,"\n")
#     print("ap",apple,"\n")
#     print("v",vudu,"\n")
#     print("g",google,"\n")

    # Fillind the general dict,so that the code is independent of any particular portal
    if(amazon_class==0):
        general = amazon.copy()
    elif(itunes_class==0):
        general = itunes.copy()
    elif(appletv_class==0):
        general = apple.copy()
    elif(google_class==0):
        general = google.copy()
    elif(vudu_class==0):
        general = vudu.copy()
        
    
#     print(general).

# Check whether the name and corresponding number of each episode is same or not
    y=0  
    z = len_
    if(appletv_class==0):
        for i in range(1,z+1):
            m=token(general[str(i)])
            n=token(apple[str(i)])            
            if(n.find(m)!=-1):
                y=y+1
            print(f"general:{m} apple:{n}")
            print(y)
            
    #print(y)        
    print("\n")
    if(google_class==0):
        for i in range(1,z+1):
            m=token(general[str(i)])
            n=token(google[str(i)])
            if(n.find(m)!=-1):
                y=y+1
            print(f"general:{m} google:{n}")
            print(y)
    
    #print(y)
    print("\n")
    if(itunes_class==0):
        for i in range(1,z+1):
            m=token(general[str(i)])
            n=token(itunes[str(i)])
            if(n.find(m)!=-1):
                y=y+1
            print(f"general:{m} itunes:{n}")
            print(y)
    
    #print(y)
    print("\n")
    if(vudu_class==0):
        for i in range(1,z+1):
            m=token(general[str(i)])
            n=token(vudu[str(i)])
            if(n.find(m)!=-1):
                y=y+1
            print(f"general:{m} vudu:{n}")
            print(y)
    
    #print(y)
    print("\n")
    if(amazon_class==0):
        for i in range(1,z+1):
            m=token(general[str(i)])
            n=token(amazon[str(i)])
            if(n.find(m)!=-1):
                y=y+1
            print(f"general:{m} amazon:{n}")
            print(y)
            

    print(f"y:{y}")
    print(f"x:{x}")
    
    #  If episode number and corresponding name are same then print properly mapped else vice versa
    if(x==5 and y==((5-(amazon_class+itunes_class+appletv_class+vudu_class+google_class))*len_)):
        print("\033[1mProperly Mapped\033[0m")
    else:
        print("\033[1mNot Properly Mapped\033[0m")




 Test Case:  3
Amazon portal is not there.
Google portal is not there.
len= 36
iTunes length is not same
Vudu length is not same
Not Properly Mapped Due to Size

 Test Case:  4
Google portal is not there.
len= 13
general:pilot apple:pilot
1
general:defiance apple:defiance
2
general:conair apple:conair
3
general:djvu apple:djvu
4
general:farewell apple:farewell
5
general:mylovely apple:mylovely
6
general:bookofsecrets apple:bookofsecrets
7
general:deadmentellnotales apple:deadmentellnotales
8
general:dangerousminds apple:dangerousminds
9
general:enemyofthestate apple:enemyofthestate
10
general:thief apple:thief
11
general:armageddon apple:armageddon
12
general:badgirls apple:badgirls
13




general:pilot itunes:pilot
14
general:defiance itunes:defiance
15
general:conair itunes:conair
16
general:djvu itunes:djvu
17
general:farewell itunes:farewell
18
general:mylovely itunes:mylovely
19
general:bookofsecrets itunes:bookofsecrets
20
general:deadmentellnotales itunes:deadmentellnotales
21
